In [1]:
!pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torch-geometric from https://files.pythonhosted.org/packages/03/9f/157e913626c1acfb3b19ce000b1a6e4e4fb177c0bc0ea0c67ca5bd714b5a/torch_geometric-2.6.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 5.8 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.8 MB/s eta 0:00:0000:01
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [12

In [2]:
python -c "import torch; print(torch.__version__)"


SyntaxError: invalid syntax (3555061209.py, line 1)

In [3]:
!python -c "import torch; print(torch.__version__)"

2.5.1+cu124


In [4]:
!pip install --upgrade pip wheel
!pip install torch torchvision torchaudio


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/c9/bc/b7db44f5f39f9d0494071bddae6880eb645970366d0a200022a1a93d57f5/pip-25.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for wheel from https://files.pythonhosted.org/packages/0b/2c/87f3254fd8ffd29e4c02732eee68a83a1d3c346ae39bc6822dcbcb697f2b/wheel-0.45.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.5.1+cu124.html
!pip install torch-geometric


Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.5 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00


In [6]:
!python -c "import torch_geometric; print('✅ PyG Installed Successfully!')"


✅ PyG Installed Successfully!


In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# ✅ Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Using device: {device}")



✅ Using device: cuda


In [2]:
# ✅ Load dataset
file_path = "IL_T600_cleaned.csv"
if not os.path.exists(file_path):
    print(f"❌ ERROR: File '{file_path}' not found! Place it in the same directory.")
else:
    df = pd.read_csv(file_path)

# ✅ Rename 'Label' to 'label' if needed
df.rename(columns={'Label': 'label'}, inplace=True)

# ✅ Separate features & labels
X = df.drop(columns=['label'])
y = df['label'].values  # 0 = Normal, 1 = Trojan

# ✅ Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features normalized.")


✅ Features normalized.


In [3]:
# ✅ Check for missing values
print("Missing values per column before handling:\n", df.isnull().sum())

# ✅ Fill NaN values with the column mean
df.fillna(df.mean(), inplace=True)

# ✅ Drop rows with any remaining NaNs (optional but safer)
df.dropna(inplace=True)

# ✅ Verify no NaNs remain
print("Missing values per column after handling:\n", df.isnull().sum())

# ✅ Now proceed with normalizing & creating KNN graph
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df.drop(columns=['label']))


Missing values per column before handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rolloff      0
spectral_entropy      0
spectral_contrast     1
dtype: int64
Missing values per column after handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rol

In [4]:
# ✅ Convert dataset into a graph using KNN
k_neighbors = 5  # Adjust based on dataset size
knn_graph = kneighbors_graph(X_scaled, k_neighbors, mode="connectivity", include_self=False)
edge_index = torch.tensor(np.array(knn_graph.nonzero()), dtype=torch.long)

# ✅ Convert to PyTorch Tensors
x_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# ✅ Create a PyTorch Geometric data object
data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor).to(device)
print("✅ Graph dataset created.")


✅ Graph dataset created.


In [5]:
# ✅ Split into training & test sets (80% train, 20% test)
train_mask = torch.rand(len(y)) < 0.8
test_mask = ~train_mask

print(f"\n✅ Training samples: {train_mask.sum().item()}")
print(f"✅ Testing samples: {test_mask.sum().item()}")



✅ Training samples: 15972
✅ Testing samples: 4028


In [6]:
# ✅ Define Graph Neural Network Model
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, output_dim=2):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# ✅ Initialize model
model = GNN(input_dim=X.shape[1]).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("✅ GNN model initialized.")


✅ GNN model initialized.


In [7]:
# ✅ Define Graph Neural Network Model
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, output_dim=2):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# ✅ Initialize model
model = GNN(input_dim=X.shape[1]).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("✅ GNN model initialized.")


✅ GNN model initialized.


In [8]:
# ✅ Train the Model
epochs = 2000
print("\n✅ Training Started...")

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    out = model(data)
    loss = criterion(out[train_mask], data.y[train_mask])

    loss.backward()
    optimizer.step()

    train_pred = out[train_mask].argmax(dim=1)
    train_acc = (train_pred == data.y[train_mask]).float().mean().item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.2%}")

print("\n✅ Training completed.")



✅ Training Started...
Epoch 1/2000, Loss: 0.6965, Train Accuracy: 48.14%
Epoch 2/2000, Loss: 0.6971, Train Accuracy: 49.76%
Epoch 3/2000, Loss: 0.6948, Train Accuracy: 48.62%
Epoch 4/2000, Loss: 0.6953, Train Accuracy: 48.47%
Epoch 5/2000, Loss: 0.6942, Train Accuracy: 48.45%
Epoch 6/2000, Loss: 0.6930, Train Accuracy: 49.44%
Epoch 7/2000, Loss: 0.6927, Train Accuracy: 52.83%
Epoch 8/2000, Loss: 0.6924, Train Accuracy: 53.43%
Epoch 9/2000, Loss: 0.6915, Train Accuracy: 53.59%
Epoch 10/2000, Loss: 0.6908, Train Accuracy: 52.06%
Epoch 11/2000, Loss: 0.6905, Train Accuracy: 50.85%
Epoch 12/2000, Loss: 0.6903, Train Accuracy: 51.14%
Epoch 13/2000, Loss: 0.6897, Train Accuracy: 52.05%
Epoch 14/2000, Loss: 0.6891, Train Accuracy: 53.58%
Epoch 15/2000, Loss: 0.6889, Train Accuracy: 54.50%
Epoch 16/2000, Loss: 0.6887, Train Accuracy: 54.58%
Epoch 17/2000, Loss: 0.6883, Train Accuracy: 54.64%
Epoch 18/2000, Loss: 0.6879, Train Accuracy: 54.14%
Epoch 19/2000, Loss: 0.6878, Train Accuracy: 53.83

In [9]:
# ✅ Save Model
torch.save(model.state_dict(), "trojan_gnn_model.pth")
print("\n✅ Model saved as 'trojan_gnn_model.pth'.")



✅ Model saved as 'trojan_gnn_model.pth'.


In [10]:
# ✅ Evaluate the Model
model.eval()
with torch.no_grad():
    out = model(data)
    test_pred = out[test_mask].argmax(dim=1)

# ✅ Compute Metrics
y_true = data.y[test_mask].cpu().numpy()
y_pred = test_pred.cpu().numpy()

accuracy = accuracy_score(y_true, y_pred) * 100
precision = precision_score(y_true, y_pred, zero_division=0) * 100
recall = recall_score(y_true, y_pred, zero_division=0) * 100
f1 = f1_score(y_true, y_pred, zero_division=0) * 100

# ✅ Print Results
print("\n🎯 Model Performance:")
print(f"✅ Accuracy: {accuracy:.2f}%")
print(f"✅ Precision: {precision:.2f}%")
print(f"✅ Recall: {recall:.2f}%")
print(f"✅ F1-score: {f1:.2f}%")



🎯 Model Performance:
✅ Accuracy: 60.30%
✅ Precision: 60.93%
✅ Recall: 57.21%
✅ F1-score: 59.01%


In [11]:
# ✅ Check sample predictions
print("\n✅ Sample Predictions:")
with torch.no_grad():
    sample_output = model(data)
    sample_pred = sample_output[:5].argmax(dim=1)

print("Predicted:", sample_pred.cpu().numpy())
print("Actual   :", data.y[:5].cpu().numpy())



✅ Sample Predictions:
Predicted: [0 0 1 0 0]
Actual   : [0 0 0 0 0]


In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.neighbors import kneighbors_graph

# ------------------------------
# 1. Generate Synthetic Time Series Data
# ------------------------------
# Let's assume we have T=500 time steps, each with D=3 features.
T = 500
D = 3

np.random.seed(42)

# Create some synthetic time series with a trend + noise
time = np.arange(T)
series = 0.5 * np.sin(0.1 * time) + 0.01 * time + 0.2 * np.random.randn(T)
# We'll make this multivariate by stacking 2 more channels
series_multi = np.stack([series,
                         series + 0.1 * np.random.randn(T),
                         0.5 * np.cos(0.1 * time) + 0.1 * np.random.randn(T)],
                        axis=1)  # shape: (T, 3)

# ------------------------------
# 2. Create Rolling Window Features
# ------------------------------
def create_rolling_windows(data, window_size=10, step=1):
    """
    data: shape (T, D)
    window_size: length of each rolling window
    step: stride between windows
    Returns: X_rolled (num_windows, window_size*D), a flattened window approach
    """
    X_rolled = []
    for start_idx in range(0, len(data) - window_size + 1, step):
        window = data[start_idx:start_idx + window_size]  # shape (window_size, D)
        X_rolled.append(window.flatten())  # flatten to 1D
    return np.array(X_rolled)

WINDOW_SIZE = 10
X_rolled = create_rolling_windows(series_multi, window_size=WINDOW_SIZE)
num_windows = X_rolled.shape[0]  # T - window_size + 1

# ------------------------------
# 3. Build Graph Edges
# ------------------------------
# (A) Temporal adjacency: connect each window i --> i+1
# We'll create edges for consecutive windows.
temporal_edges = []
for i in range(num_windows - 1):
    temporal_edges.append([i, i+1])  # forward edge
    temporal_edges.append([i+1, i])  # backward edge (if you want undirected)
temporal_edges = np.array(temporal_edges).T  # shape (2, ?)

# (B) KNN adjacency: connect each window to k similar windows based on feature space
k_neighbors = 5  # can adjust
knn_graph = kneighbors_graph(X_rolled, k_neighbors, mode='connectivity', include_self=False)
knn_coo = knn_graph.tocoo()
knn_edges = np.vstack([knn_coo.row, knn_coo.col])  # shape (2, n_edges)

# Combine edges (temporal + KNN)
all_edges = np.concatenate([temporal_edges, knn_edges], axis=1)

# Convert to a PyTorch long tensor
edge_index = torch.tensor(all_edges, dtype=torch.long)

# ------------------------------
# 4. Create Labels for Supervised Learning
# ------------------------------
# Suppose we want to classify each window as "1" if the *average* in that window > 0,
# else "0". This is just a synthetic classification target.

window_averages = X_rolled.mean(axis=1)
labels = (window_averages > 0).astype(int)

# ------------------------------
# 5. Build PyTorch Geometric Data Object
# ------------------------------
x_tensor = torch.tensor(X_rolled, dtype=torch.float32)  # Node features
y_tensor = torch.tensor(labels, dtype=torch.long)        # Node labels

data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor)

# ------------------------------
# 6. Train/Test Split
# ------------------------------
# Let's do a simple random split of the windows
train_mask = torch.rand(num_windows) < 0.8
test_mask = ~train_mask

# ------------------------------
# 7. Define GCN Model
# ------------------------------
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=32, output_dim=2):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = nn.ReLU()(x)
        x = self.conv2(x, edge_index)
        return x

model = GNN(input_dim=X_rolled.shape[1], hidden_dim=64, output_dim=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# ------------------------------
# 8. Training
# ------------------------------
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    out = model(data)   # shape [num_windows, 2]
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # Compute training accuracy
    train_pred = out[train_mask].argmax(dim=1)
    train_acc = (train_pred == data.y[train_mask]).float().mean().item()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item():.4f} | Train Acc: {train_acc:.2%}")

# ------------------------------
# 9. Evaluation
# ------------------------------
model.eval()
with torch.no_grad():
    out = model(data)
    test_pred = out[test_mask].argmax(dim=1)
    test_acc = (test_pred == data.y[test_mask]).float().mean().item()

print(f"\nTest Accuracy: {test_acc:.2%}")


Epoch 10/100 | Loss: 0.0146 | Train Acc: 99.75%
Epoch 20/100 | Loss: 0.0042 | Train Acc: 100.00%
Epoch 30/100 | Loss: 0.0021 | Train Acc: 100.00%
Epoch 40/100 | Loss: 0.0012 | Train Acc: 100.00%
Epoch 50/100 | Loss: 0.0008 | Train Acc: 100.00%
Epoch 60/100 | Loss: 0.0005 | Train Acc: 100.00%
Epoch 70/100 | Loss: 0.0004 | Train Acc: 100.00%
Epoch 80/100 | Loss: 0.0003 | Train Acc: 100.00%
Epoch 90/100 | Loss: 0.0002 | Train Acc: 100.00%
Epoch 100/100 | Loss: 0.0002 | Train Acc: 100.00%

Test Accuracy: 100.00%


In [18]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# ------------------------------
# 1. Check for GPU
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Using device: {device}")

# ------------------------------
# 2. Load the Dataset
# ------------------------------
file_path = "IL_T600_cleaned.csv"
if not os.path.exists(file_path):
    print(f"❌ ERROR: File '{file_path}' not found! Place it in the same directory.")
    exit(1)

df = pd.read_csv(file_path)

# Rename 'Label' to 'label' if needed
df.rename(columns={'Label': 'label'}, inplace=True)

# ------------------------------
# 3. Handle Missing Values
# ------------------------------
print("Missing values per column before handling:\n", df.isnull().sum())

# Fill NaN values with column mean
df.fillna(df.mean(), inplace=True)

# Optional: Drop rows with any remaining NaNs
df.dropna(inplace=True)

print("Missing values per column after handling:\n", df.isnull().sum())

# ------------------------------
# 4. Separate Features & Labels
# ------------------------------
if 'label' not in df.columns:
    raise ValueError("❌ 'label' column is missing from the CSV. Please check your data.")

X = df.drop(columns=['label'])
y = df['label'].values  # 0 = Normal, 1 = Trojan (assuming binary)

# ------------------------------
# 5. Normalize Features
# ------------------------------
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features normalized.")

# ------------------------------
# 6. Build the Graph Edges
# ------------------------------
# (A) Temporal adjacency: connect each time step i -> i+1 (and i+1 -> i for undirected)
temporal_edges = []
num_samples = len(X_scaled)
for i in range(num_samples - 1):
    temporal_edges.append([i, i+1])
    temporal_edges.append([i+1, i])  # comment out if you prefer a directed chain

temporal_edges = np.array(temporal_edges).T  # shape (2, 2*(num_samples-1))

# (B) KNN adjacency
k_neighbors = 5  # Adjust based on dataset size
knn_graph = kneighbors_graph(X_scaled, k_neighbors, mode="connectivity", include_self=False)
knn_coo = knn_graph.tocoo()
knn_edges = np.vstack([knn_coo.row, knn_coo.col])  # shape (2, number_of_knn_edges)

# Combine edges
all_edges = np.concatenate([temporal_edges, knn_edges], axis=1)
edge_index = torch.tensor(all_edges, dtype=torch.long)

# ------------------------------
# 7. Create PyTorch Geometric Data
# ------------------------------
x_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor).to(device)
print("✅ Graph dataset created with temporal + KNN edges.")

# ------------------------------
# 8. Train/Test Split
# ------------------------------
# Randomly assign ~80% of nodes to training, 20% to testing
mask = torch.rand(num_samples).to(device)
train_mask = mask < 0.8
test_mask = ~train_mask

print(f"\n✅ Training samples: {train_mask.sum().item()}")
print(f"✅ Testing samples: {test_mask.sum().item()}")

# ------------------------------
# 9. Define GNN Model
# ------------------------------
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, output_dim=2):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# Initialize model, optimizer, loss
model = GNN(input_dim=X.shape[1], hidden_dim=256, output_dim=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
print("✅ GNN model initialized.")

# ------------------------------
# 10. Train the Model
# ------------------------------
epochs = 2000
print("\n✅ Training Started...")

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    out = model(data)  # shape: [num_samples, 2]
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # Compute training accuracy
    train_pred = out[train_mask].argmax(dim=1)
    train_acc = (train_pred == data.y[train_mask]).float().mean().item()

    if (epoch + 1) % 100 == 0:  # Print every 100 epochs
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.2%}")

print("\n✅ Training completed.")

# ------------------------------
# 11. Save the Model
# ------------------------------
torch.save(model.state_dict(), "trojan_gnn_model.pth")
print("\n✅ Model saved as 'trojan_gnn_model.pth'.")

# ------------------------------
# 12. Evaluate the Model
# ------------------------------
model.eval()
with torch.no_grad():
    out = model(data)
    test_pred = out[test_mask].argmax(dim=1)

# Compute Metrics
y_true = data.y[test_mask].cpu().numpy()
y_pred = test_pred.cpu().numpy()

accuracy = accuracy_score(y_true, y_pred) * 100
precision = precision_score(y_true, y_pred, zero_division=0) * 100
recall = recall_score(y_true, y_pred, zero_division=0) * 100
f1 = f1_score(y_true, y_pred, zero_division=0) * 100

# Print Results
print("\n🎯 Model Performance on Test Nodes:")
print(f"✅ Accuracy:  {accuracy:.2f}%")
print(f"✅ Precision: {precision:.2f}%")
print(f"✅ Recall:    {recall:.2f}%")
print(f"✅ F1-score:  {f1:.2f}%")



✅ Using device: cuda
Missing values per column before handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rolloff      0
spectral_entropy      0
spectral_contrast     1
dtype: int64
Missing values per column after handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatn

In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# ------------------------------
# 1. Check for GPU
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Using device: {device}")

# ------------------------------
# 2. Load and Preprocess the Dataset
# ------------------------------
file_path = "IL_T600_cleaned.csv"
if not os.path.exists(file_path):
    print(f"❌ ERROR: File '{file_path}' not found! Place it in the same directory.")
    raise FileNotFoundError(file_path)

df = pd.read_csv(file_path)

# Rename 'Label' to 'label' if needed
df.rename(columns={'Label': 'label'}, inplace=True)

# Check for missing values
print("Missing values per column before handling:\n", df.isnull().sum())

# Fill NaN values with mean
df.fillna(df.mean(), inplace=True)

# (Optional) Drop any rows still containing NaN
df.dropna(inplace=True)

print("Missing values per column after handling:\n", df.isnull().sum())

# Ensure the 'label' column exists
if 'label' not in df.columns:
    raise ValueError("❌ 'label' column is missing from the CSV. Please check your data.")

# Separate features (X) and labels (y)
X = df.drop(columns=['label'])
y = df['label'].values  # 0 = Normal, 1 = Trojan (assuming binary classification)

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features normalized.")

# ------------------------------
# 3. Build Graph Edges
# ------------------------------
num_samples = len(X_scaled)

# (A) Temporal edges: connect i -> i+1 (and i+1 -> i to make it undirected)
temporal_edges = []
for i in range(num_samples - 1):
    temporal_edges.append([i, i+1])
    temporal_edges.append([i+1, i])

temporal_edges = np.array(temporal_edges).T  # shape: (2, 2*(num_samples - 1))

# (B) KNN edges: connect each sample to its k nearest neighbors in feature space
k_neighbors = 5  # Adjust based on dataset size/needs
knn_graph = kneighbors_graph(X_scaled, k_neighbors, mode="connectivity", include_self=False)
knn_coo = knn_graph.tocoo()
knn_edges = np.vstack([knn_coo.row, knn_coo.col])  # shape: (2, num_knn_edges)

# Combine temporal and KNN edges
all_edges = np.concatenate([temporal_edges, knn_edges], axis=1)
edge_index = torch.tensor(all_edges, dtype=torch.long)

# ------------------------------
# 4. Create PyTorch Geometric Data
# ------------------------------
x_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor).to(device)
print("✅ Graph dataset created with temporal + KNN edges.")

# ------------------------------
# 5. Chronological Split (Train = first 80%, Test = last 20%)
# ------------------------------
split_point = int(0.8 * num_samples)  # 80% cutoff
# Create boolean masks for each node (row)
train_mask = torch.zeros(num_samples, dtype=torch.bool).to(device)
train_mask[:split_point] = True
test_mask = ~train_mask

print(f"\n✅ Training samples: {train_mask.sum().item()}")
print(f"✅ Testing samples: {test_mask.sum().item()}")

# ------------------------------
# 6. Define a Deeper GNN Model (3 GCN Layers + Dropout)
# ------------------------------
class DeeperGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=2, dropout_p=0.2):
        super(DeeperGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First layer
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)

        # Second layer
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)

        # Third layer (logits)
        x = self.conv3(x, edge_index)

        return x

# Initialize model
model = DeeperGNN(input_dim=X.shape[1], hidden_dim=128, output_dim=2, dropout_p=0.3).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

print("✅ Deeper GNN model initialized.")

# ------------------------------
# 7. Train the Model
# ------------------------------
epochs = 500
print("\n✅ Training started...")

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    out = model(data)  # shape: [num_samples, 2]
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # Compute training accuracy
    train_pred = out[train_mask].argmax(dim=1)
    train_acc = (train_pred == data.y[train_mask]).float().mean().item()

    if (epoch + 1) % 50 == 0:  # Print every 50 epochs
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.2%}")

print("\n✅ Training completed.")

# ------------------------------
# 8. Save the Model
# ------------------------------
torch.save(model.state_dict(), "trojan_gnn_model_deeper.pth")
print("\n✅ Model saved as 'trojan_gnn_model_deeper.pth'.")

# ------------------------------
# 9. Evaluate the Model
# ------------------------------
model.eval()
with torch.no_grad():
    out = model(data)
    test_pred = out[test_mask].argmax(dim=1)

# Compute Metrics
y_true = data.y[test_mask].cpu().numpy()
y_pred = test_pred.cpu().numpy()

accuracy = accuracy_score(y_true, y_pred) * 100
precision = precision_score(y_true, y_pred, zero_division=0) * 100
recall = recall_score(y_true, y_pred, zero_division=0) * 100
f1 = f1_score(y_true, y_pred, zero_division=0) * 100

print("\n🎯 Model Performance on Test Set:")
print(f"✅ Accuracy:  {accuracy:.2f}%")
print(f"✅ Precision: {precision:.2f}%")
print(f"✅ Recall:    {recall:.2f}%")
print(f"✅ F1-score:  {f1:.2f}%")



✅ Using device: cuda
Missing values per column before handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rolloff      0
spectral_entropy      0
spectral_contrast     1
dtype: int64
Missing values per column after handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatn

In [15]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# ------------------------------
# 1. Check for GPU
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Using device: {device}")

# ------------------------------
# 2. Load & Clean the CSV
# ------------------------------
file_path = "IL_T600_cleaned.csv"
if not os.path.exists(file_path):
    print(f"❌ ERROR: File '{file_path}' not found! Place it in the same directory.")
    raise FileNotFoundError(file_path)

df = pd.read_csv(file_path)

# Rename 'Label' to 'label' if needed
df.rename(columns={'Label': 'label'}, inplace=True)

print("Missing values per column before handling:\n", df.isnull().sum())

# Fill NaN with column mean
df.fillna(df.mean(), inplace=True)

# (Optional) Drop rows with any remaining NaN
df.dropna(inplace=True)

print("Missing values per column after handling:\n", df.isnull().sum())

if 'label' not in df.columns:
    raise ValueError("❌ 'label' column is missing from the CSV. Please check your data.")

# ------------------------------
# 3. Separate Features (X) and Labels (y)
# ------------------------------
X = df.drop(columns=['label'])
y = df['label'].values  # e.g. 0 = Normal, 1 = Trojan

# Normalize Features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features normalized.")

num_samples = len(X_scaled)

# ------------------------------
# 4. Build Graph Edges: Temporal + KNN
# ------------------------------
# A) Temporal Edges: connect each row i->i+1 and i+1->i
temporal_edges = []
for i in range(num_samples - 1):
    temporal_edges.append([i, i+1])
    temporal_edges.append([i+1, i])  # Undirected

temporal_edges = np.array(temporal_edges).T  # shape (2, 2*(num_samples - 1))

# B) KNN Edges
k_neighbors = 5
knn_graph = kneighbors_graph(X_scaled, k_neighbors, mode="connectivity", include_self=False)
knn_coo = knn_graph.tocoo()
knn_edges = np.vstack([knn_coo.row, knn_coo.col])  # shape (2, num_knn_edges)

# Combine edges
all_edges = np.concatenate([temporal_edges, knn_edges], axis=1)
edge_index = torch.tensor(all_edges, dtype=torch.long)

# ------------------------------
# 5. Create PyTorch Geometric Data
# ------------------------------
x_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor).to(device)
print("✅ Graph dataset created with temporal + KNN edges.")

# ------------------------------
# 6. Chronological Split (80% Train, 20% Test)
# ------------------------------
split_point = int(0.8 * num_samples)  # 80% cutoff
train_mask = torch.zeros(num_samples, dtype=torch.bool).to(device)
train_mask[:split_point] = True
test_mask = ~train_mask

print(f"\n✅ Training samples: {train_mask.sum().item()}")
print(f"✅ Testing samples: {test_mask.sum().item()}")

# ------------------------------
# 7. Define a Deeper GNN (3 GCN Layers + Dropout)
# ------------------------------
class DeeperGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=2, dropout_p=0.3):
        super(DeeperGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # First layer
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)

        # Second layer
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)

        # Third layer (logits)
        x = self.conv3(x, edge_index)
        return x

model = DeeperGNN(input_dim=X.shape[1], hidden_dim=128, output_dim=2, dropout_p=0.3).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

print("✅ Deeper GNN model initialized.")

# ------------------------------
# 8. Train the Model
# ------------------------------
epochs = 500
print("\n✅ Training started...")

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    out = model(data)  # shape: [num_samples, 2]
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # Compute training accuracy
    with torch.no_grad():
        train_pred = out[train_mask].argmax(dim=1)
        train_acc = (train_pred == data.y[train_mask]).float().mean().item()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.2%}")

print("\n✅ Training completed.")

# ------------------------------
# 9. Save the Model
# ------------------------------
torch.save(model.state_dict(), "trojan_gnn_model_deeper.pth")
print("\n✅ Model saved as 'trojan_gnn_model_deeper.pth'.")

# ------------------------------
# 10. Evaluate the Model
# ------------------------------
model.eval()
with torch.no_grad():
    out = model(data)
    test_pred = out[test_mask].argmax(dim=1)

# Convert to NumPy for metrics
y_true = data.y[test_mask].cpu().numpy()
y_pred = test_pred.cpu().numpy()

accuracy = accuracy_score(y_true, y_pred) * 100
precision = precision_score(y_true, y_pred, zero_division=0) * 100
recall = recall_score(y_true, y_pred, zero_division=0) * 100
f1 = f1_score(y_true, y_pred, zero_division=0) * 100

print("\n🎯 Model Performance on Test Set:")
print(f"✅ Accuracy:  {accuracy:.2f}%")
print(f"✅ Precision: {precision:.2f}%")
print(f"✅ Recall:    {recall:.2f}%")
print(f"✅ F1-score:  {f1:.2f}%")



✅ Using device: cuda
Missing values per column before handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rolloff      0
spectral_entropy      0
spectral_contrast     1
dtype: int64
Missing values per column after handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatn

In [16]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# ------------------------------
# 1. Check for GPU
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Using device: {device}")

# ------------------------------
# 2. Load & Clean the CSV
# ------------------------------
file_path = "IL_T600_cleaned.csv"
if not os.path.exists(file_path):
    print(f"❌ ERROR: File '{file_path}' not found! Place it in the same directory.")
    raise FileNotFoundError(file_path)

df = pd.read_csv(file_path)

# Rename 'Label' to 'label' if needed
df.rename(columns={'Label': 'label'}, inplace=True)

# Basic missing-value handling
print("Missing values per column before handling:\n", df.isnull().sum())
df.fillna(df.mean(), inplace=True)
df.dropna(inplace=True)  # optional final drop of any row with remaining NaN
print("Missing values per column after handling:\n", df.isnull().sum())

if 'label' not in df.columns:
    raise ValueError("❌ 'label' column is missing from the CSV. Please check your data.")

# ------------------------------
# 3. Separate Features (X) & Labels (y)
# ------------------------------
X = df.drop(columns=['label'])
y = df['label'].values  # 0 = Normal, 1 = Trojan (assuming binary classification)

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features normalized.")

num_samples = len(X_scaled)

# ------------------------------
# 4. Build Graph Edges: Temporal + KNN
# ------------------------------
# (A) Temporal edges: connect each row i->i+1 and i+1->i
temporal_edges = []
for i in range(num_samples - 1):
    temporal_edges.append([i, i+1])
    temporal_edges.append([i+1, i])  # Undirected

temporal_edges = np.array(temporal_edges).T  # shape (2, 2*(num_samples - 1))

# (B) KNN edges
k_neighbors = 5
knn_graph = kneighbors_graph(X_scaled, k_neighbors, mode="connectivity", include_self=False)
knn_coo = knn_graph.tocoo()
knn_edges = np.vstack([knn_coo.row, knn_coo.col])  # shape (2, num_knn_edges)

# Combine edges
all_edges = np.concatenate([temporal_edges, knn_edges], axis=1)
edge_index = torch.tensor(all_edges, dtype=torch.long)

# ------------------------------
# 5. Create PyTorch Geometric Data
# ------------------------------
x_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor).to(device)
print("✅ Graph dataset created with temporal + KNN edges.")

# ------------------------------
# 6. Chronological Split
#     - Train: first 70%
#     - Val: next 10%
#     - Test: final 20%
# ------------------------------
train_cut = int(0.7 * num_samples)
val_cut = int(0.8 * num_samples)

train_mask = torch.zeros(num_samples, dtype=torch.bool).to(device)
val_mask = torch.zeros(num_samples, dtype=torch.bool).to(device)
test_mask = torch.zeros(num_samples, dtype=torch.bool).to(device)

train_mask[:train_cut] = True
val_mask[train_cut:val_cut] = True
test_mask[val_cut:] = True

print(f"\n✅ Training samples: {train_mask.sum().item()}")
print(f"✅ Validation samples: {val_mask.sum().item()}")
print(f"✅ Testing samples: {test_mask.sum().item()}")

# ------------------------------
# 7. Define a Deeper GNN (3 GCN Layers + Dropout)
# ------------------------------
class DeeperGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=2, dropout_p=0.3):
        super(DeeperGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # First layer
        x = self.conv1(x, edge_index).relu()
        x = self.dropout(x)

        # Second layer
        x = self.conv2(x, edge_index).relu()
        x = self.dropout(x)

        # Third layer (logits)
        x = self.conv3(x, edge_index)
        return x

model = DeeperGNN(input_dim=X.shape[1], hidden_dim=128, output_dim=2, dropout_p=0.3).to(device)

# ------------------------------
# 8. (Optional) Weighted Loss for Class Imbalance
#    If Trojan is rare, try weighting the classes.
#    For example, weight= [1.0, 5.0] if Trojan is 5x less common than Normal.
# ------------------------------
# Let's do a quick check of class distribution:
trojan_count = (y == 1).sum()
normal_count = (y == 0).sum()
ratio = normal_count / float(trojan_count) if trojan_count != 0 else 1.0
print(f"Class Distribution -> Normal: {normal_count}, Trojan: {trojan_count}, Ratio: {ratio:.2f}")

# Adjust the weighting factor as needed:
weight_tensor = torch.tensor([1.0, max(1.0, ratio)], dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=weight_tensor)
# If you're sure you don't want weighting, comment above lines and do:
# criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
print("\n✅ Deeper GNN model initialized with class weighting and Adam optimizer.")

# ------------------------------
# 9. Train the Model (with Early Validation)
# ------------------------------
epochs = 500
print("\n✅ Training started...")

best_val_loss = float('inf')
best_epoch = 0
patience = 50  # If val_loss doesn't improve for 50 epochs, stop early

for epoch in range(1, epochs+1):
    # ---- Train Step ----
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss_train = criterion(out[train_mask], data.y[train_mask])
    loss_train.backward()
    optimizer.step()

    # ---- Validation Step ----
    model.eval()
    with torch.no_grad():
        out_val = model(data)
        loss_val = criterion(out_val[val_mask], data.y[val_mask])
    
    # Print status
    if epoch % 50 == 0:
        train_pred = out[train_mask].argmax(dim=1)
        train_acc = (train_pred == data.y[train_mask]).float().mean().item()
        print(f"Epoch {epoch}/{epochs}, "
              f"Train Loss: {loss_train.item():.4f}, "
              f"Val Loss: {loss_val.item():.4f}, "
              f"Train Acc: {train_acc:.2%}")

    # Early stopping check
    if loss_val.item() < best_val_loss:
        best_val_loss = loss_val.item()
        best_epoch = epoch
        best_state_dict = model.state_dict()  # Save model weights
    else:
        # If no improvement for 'patience' epochs, stop
        if epoch - best_epoch >= patience:
            print(f"\nEarly stopping at epoch {epoch}, "
                  f"no improvement since epoch {best_epoch}.")
            break

# Load the best model weights
model.load_state_dict(best_state_dict)
print("\n✅ Training completed, best model from epoch:", best_epoch)

# ------------------------------
# 10. Save the Best Model
# ------------------------------
torch.save(model.state_dict(), "trojan_gnn_model_deeper.pth")
print("\n✅ Model saved as 'trojan_gnn_model_deeper.pth'.")

# ------------------------------
# 11. Evaluate on Test Set
# ------------------------------
model.eval()
with torch.no_grad():
    out_test = model(data)
    test_pred = out_test[test_mask].argmax(dim=1)

# Convert to NumPy for metrics
y_true = data.y[test_mask].cpu().numpy()
y_pred = test_pred.cpu().numpy()

accuracy = accuracy_score(y_true, y_pred) * 100
precision = precision_score(y_true, y_pred, zero_division=0) * 100
recall = recall_score(y_true, y_pred, zero_division=0) * 100
f1 = f1_score(y_true, y_pred, zero_division=0) * 100

print("\n🎯 Model Performance on Test Set:")
print(f"✅ Accuracy:  {accuracy:.2f}%")
print(f"✅ Precision: {precision:.2f}%")
print(f"✅ Recall:    {recall:.2f}%")
print(f"✅ F1-score:  {f1:.2f}%")



✅ Using device: cuda
Missing values per column before handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rolloff      0
spectral_entropy      0
spectral_contrast     1
dtype: int64
Missing values per column after handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatn

In [17]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# ------------------------------
# 1. Check for GPU
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Using device: {device}")

# ------------------------------
# 2. Load & Clean the CSV
# ------------------------------
file_path = "IL_T600_cleaned.csv"
if not os.path.exists(file_path):
    raise FileNotFoundError(f"❌ ERROR: File '{file_path}' not found!")

df = pd.read_csv(file_path)

# Rename 'Label' to 'label' if needed
df.rename(columns={'Label': 'label'}, inplace=True)

# Basic missing-value handling
print("Missing values per column before handling:\n", df.isnull().sum())
df.fillna(df.mean(), inplace=True)
df.dropna(inplace=True)  # optional final drop of any row with remaining NaN
print("Missing values per column after handling:\n", df.isnull().sum())

if 'label' not in df.columns:
    raise ValueError("❌ 'label' column is missing from the CSV. Please check your data.")

# ------------------------------
# 3. Separate Features (X) & Labels (y)
# ------------------------------
X = df.drop(columns=['label'])
y = df['label'].values  # 0 = Normal, 1 = Trojan

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features normalized.")

num_samples = len(X_scaled)

# ------------------------------
# 4. Build Graph Edges: Temporal + KNN
# ------------------------------
# (A) Temporal edges (undirected): connect i->i+1 and i+1->i
temporal_edges = []
for i in range(num_samples - 1):
    temporal_edges.append([i, i+1])
    temporal_edges.append([i+1, i])

temporal_edges = np.array(temporal_edges).T  # shape (2, 2*(num_samples - 1))

# (B) KNN edges (smaller k to avoid over-connection)
k_neighbors = 3
knn_graph = kneighbors_graph(X_scaled, k_neighbors, mode="connectivity", include_self=False)
knn_coo = knn_graph.tocoo()
knn_edges = np.vstack([knn_coo.row, knn_coo.col])  # shape: (2, num_knn_edges)

# Combine edges
all_edges = np.concatenate([temporal_edges, knn_edges], axis=1)
edge_index = torch.tensor(all_edges, dtype=torch.long)

# ------------------------------
# 5. Create PyTorch Geometric Data
# ------------------------------
x_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor).to(device)
print("✅ Graph dataset created with temporal + KNN edges.")

# ------------------------------
# 6. Chronological Split (70% / 10% / 20%)
# ------------------------------
train_cut = int(0.7 * num_samples)
val_cut = int(0.8 * num_samples)

train_mask = torch.zeros(num_samples, dtype=torch.bool).to(device)
val_mask = torch.zeros(num_samples, dtype=torch.bool).to(device)
test_mask = torch.zeros(num_samples, dtype=torch.bool).to(device)

train_mask[:train_cut] = True
val_mask[train_cut:val_cut] = True
test_mask[val_cut:] = True

print(f"\n✅ Training samples: {train_mask.sum().item()}")
print(f"✅ Validation samples: {val_mask.sum().item()}")
print(f"✅ Testing samples: {test_mask.sum().item()}")

# Optional: check distribution
trojan_count = (y == 1).sum()
normal_count = (y == 0).sum()
print(f"Class Distribution -> Normal: {normal_count}, Trojan: {trojan_count}")

# ------------------------------
# 7. Define a Deeper GNN (3 GCN Layers + Smaller Dropout)
# ------------------------------
class DeeperGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=2, dropout_p=0.1):
        super(DeeperGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # First layer
        x = self.conv1(x, edge_index)
        x = nn.ReLU()(x)
        x = self.dropout(x)

        # Second layer
        x = self.conv2(x, edge_index)
        x = nn.ReLU()(x)
        x = self.dropout(x)

        # Third layer (logits)
        x = self.conv3(x, edge_index)
        return x

model = DeeperGNN(input_dim=X.shape[1], hidden_dim=128, output_dim=2, dropout_p=0.1).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()  # Balanced dataset? Then no weighting needed

print("\n✅ Deeper GNN model initialized.")

# ------------------------------
# 8. Train the Model (No Hard Early Stopping)
#    We'll do a fixed number of epochs (e.g., 300)
#    and track "best model" based on validation accuracy
# ------------------------------
epochs = 300
print("\n✅ Training started...")

best_val_acc = 0.0
best_epoch = 0
best_state_dict = None

for epoch in range(1, epochs + 1):
    # ---- Train Step ----
    model.train()
    optimizer.zero_grad()
    
    out = model(data)
    loss_train = criterion(out[train_mask], data.y[train_mask])
    loss_train.backward()
    optimizer.step()

    # ---- Validation Step ----
    model.eval()
    with torch.no_grad():
        out_val = model(data)[val_mask]
        val_labels = data.y[val_mask]
        loss_val = criterion(out_val, val_labels)

        # Compute validation accuracy
        val_pred = out_val.argmax(dim=1)
        val_acc = (val_pred == val_labels).float().mean().item()

    # Print progress every 20 epochs
    if epoch % 20 == 0:
        # Train accuracy (optional to check)
        with torch.no_grad():
            train_pred = out[train_mask].argmax(dim=1)
            train_acc = (train_pred == data.y[train_mask]).float().mean().item()
        print(f"Epoch {epoch}/{epochs}, "
              f"Train Loss: {loss_train.item():.4f}, "
              f"Val Loss: {loss_val.item():.4f}, "
              f"Train Acc: {train_acc:.2%}, "
              f"Val Acc: {val_acc:.2%}")

    # Save best model (based on val_acc)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch
        best_state_dict = model.state_dict()

print(f"\n✅ Training completed. Best validation accuracy = {best_val_acc:.2%} at epoch {best_epoch}.")

# Load the best model for final evaluation
if best_state_dict is not None:
    model.load_state_dict(best_state_dict)

# ------------------------------
# 9. Save the Best Model
# ------------------------------
torch.save(model.state_dict(), "trojan_gnn_model_deeper.pth")
print("\n✅ Best model saved as 'trojan_gnn_model_deeper.pth'.")

# ------------------------------
# 10. Evaluate on Test Set
# ------------------------------
model.eval()
with torch.no_grad():
    out_test = model(data)[test_mask]
    test_pred = out_test.argmax(dim=1)

# Convert to NumPy for metrics
y_true = data.y[test_mask].cpu().numpy()
y_pred = test_pred.cpu().numpy()

accuracy = accuracy_score(y_true, y_pred) * 100
precision = precision_score(y_true, y_pred, zero_division=0) * 100
recall = recall_score(y_true, y_pred, zero_division=0) * 100
f1 = f1_score(y_true, y_pred, zero_division=0) * 100

print("\n🎯 Model Performance on Test Set (Best Model):")
print(f"✅ Accuracy:  {accuracy:.2f}%")
print(f"✅ Precision: {precision:.2f}%")
print(f"✅ Recall:    {recall:.2f}%")
print(f"✅ F1-score:  {f1:.2f}%")



✅ Using device: cuda
Missing values per column before handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rolloff      0
spectral_entropy      0
spectral_contrast     1
dtype: int64
Missing values per column after handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatn

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# ------------------------------
# 1. Check for GPU
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Using device: {device}")

# ------------------------------
# 2. Load the Dataset
# ------------------------------
file_path = "IL_T600_cleaned.csv"
if not os.path.exists(file_path):
    print(f"❌ ERROR: File '{file_path}' not found! Place it in the same directory.")
    exit(1)

df = pd.read_csv(file_path)

# Rename 'Label' to 'label' if needed
df.rename(columns={'Label': 'label'}, inplace=True)

# ------------------------------
# 3. Handle Missing Values
# ------------------------------
print("Missing values per column before handling:\n", df.isnull().sum())

# Fill NaN values with column mean
df.fillna(df.mean(), inplace=True)

# Optional: Drop rows with any remaining NaNs
df.dropna(inplace=True)

print("Missing values per column after handling:\n", df.isnull().sum())

# ------------------------------
# 4. Separate Features & Labels
# ------------------------------
if 'label' not in df.columns:
    raise ValueError("❌ 'label' column is missing from the CSV. Please check your data.")

X = df.drop(columns=['label'])
y = df['label'].values  # 0 = Normal, 1 = Trojan (assuming binary)

# ------------------------------
# 5. Normalize Features
# ------------------------------
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features normalized.")

# ------------------------------
# 6. Build the Graph Edges
# ------------------------------
# (A) Temporal adjacency: connect each time step i -> i+1 (and i+1 -> i for undirected)
temporal_edges = []
num_samples = len(X_scaled)
for i in range(num_samples - 1):
    temporal_edges.append([i, i+1])
    temporal_edges.append([i+1, i])  # comment out if you prefer a directed chain

temporal_edges = np.array(temporal_edges).T  # shape (2, 2*(num_samples-1))

# (B) KNN adjacency
k_neighbors = 5  # Adjust based on dataset size
knn_graph = kneighbors_graph(X_scaled, k_neighbors, mode="connectivity", include_self=False)
knn_coo = knn_graph.tocoo()
knn_edges = np.vstack([knn_coo.row, knn_coo.col])  # shape (2, number_of_knn_edges)

# Combine edges
all_edges = np.concatenate([temporal_edges, knn_edges], axis=1)
edge_index = torch.tensor(all_edges, dtype=torch.long)

# ------------------------------
# 7. Create PyTorch Geometric Data
# ------------------------------
x_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor).to(device)
print("✅ Graph dataset created with temporal + KNN edges.")

# ------------------------------
# 8. Train/Test Split
# ------------------------------
# Randomly assign ~80% of nodes to training, 20% to testing
mask = torch.rand(num_samples).to(device)
train_mask = mask < 0.8
test_mask = ~train_mask

print(f"\n✅ Training samples: {train_mask.sum().item()}")
print(f"✅ Testing samples: {test_mask.sum().item()}")

# ------------------------------
# 9. Define GNN Model
# ------------------------------
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, output_dim=2):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# Initialize model, optimizer, loss
model = GNN(input_dim=X.shape[1], hidden_dim=256, output_dim=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
print("✅ GNN model initialized.")

# ------------------------------
# 10. Train the Model
# ------------------------------
epochs = 2000
print("\n✅ Training Started...")

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    out = model(data)  # shape: [num_samples, 2]
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # Compute training accuracy
    train_pred = out[train_mask].argmax(dim=1)
    train_acc = (train_pred == data.y[train_mask]).float().mean().item()

    if (epoch + 1) % 100 == 0:  # Print every 100 epochs
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.2%}")

print("\n✅ Training completed.")

# ------------------------------
# 11. Save the Model
# ------------------------------
torch.save(model.state_dict(), "trojan_gnn_model.pth")
print("\n✅ Model saved as 'trojan_gnn_model.pth'.")

# ------------------------------
# 12. Evaluate the Model
# ------------------------------
model.eval()
with torch.no_grad():
    out = model(data)
    test_pred = out[test_mask].argmax(dim=1)

# Compute Metrics
y_true = data.y[test_mask].cpu().numpy()
y_pred = test_pred.cpu().numpy()

accuracy = accuracy_score(y_true, y_pred) * 100
precision = precision_score(y_true, y_pred, zero_division=0) * 100
recall = recall_score(y_true, y_pred, zero_division=0) * 100
f1 = f1_score(y_true, y_pred, zero_division=0) * 100

# Print Results
print("\n🎯 Model Performance on Test Nodes:")
print(f"✅ Accuracy:  {accuracy:.2f}%")
print(f"✅ Precision: {precision:.2f}%")
print(f"✅ Recall:    {recall:.2f}%")
print(f"✅ F1-score:  {f1:.2f}%")



✅ Using device: cuda
Missing values per column before handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rolloff      0
spectral_entropy      0
spectral_contrast     1
dtype: int64
Missing values per column after handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatn

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import os

# ------------------------------
# 1. Load and Validate Dataset
# ------------------------------
file_path = "IL_T600_cleaned.csv"
if not os.path.exists(file_path):
    print(f"❌ ERROR: File '{file_path}' not found! Place it in the same directory.")
    exit(1)

df = pd.read_csv(file_path)
df.rename(columns={'Label': 'label'}, inplace=True)

# ------------------------------
# 2. Handle Missing Values
# ------------------------------
print("Missing values before handling:\n", df.isnull().sum())
df.fillna(df.mean(), inplace=True)
df.dropna(inplace=True)
print("Missing values after handling:\n", df.isnull().sum())

# ------------------------------
# 3. Separate Features and Labels
# ------------------------------
if 'label' not in df.columns:
    raise ValueError("❌ 'label' column is missing from the CSV. Please check your data.")

X = df.drop(columns=['label'])
y = df['label'].values  # 0 = Clean, 1 = Trojan

# ------------------------------
# 4. Normalize Features
# ------------------------------
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features normalized.")

# ------------------------------
# 5. Isolation Forest Model
# ------------------------------
print("\n✅ Training Isolation Forest...")
iso_forest = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)
iso_forest.fit(X_scaled)

# ------------------------------
# 6. Predict and Evaluate
# ------------------------------
y_pred = iso_forest.predict(X_scaled)
# Isolation Forest output: -1 = anomaly → Trojan (1), 1 = normal → Clean (0)
y_pred_mapped = np.array([1 if val == -1 else 0 for val in y_pred])

# Compute Metrics
accuracy = accuracy_score(y, y_pred_mapped) * 100
precision = precision_score(y, y_pred_mapped, zero_division=0) * 100
recall = recall_score(y, y_pred_mapped, zero_division=0) * 100
f1 = f1_score(y, y_pred_mapped, zero_division=0) * 100

# ------------------------------
# 7. Print Results
# ------------------------------
print("\n🎯 Isolation Forest Performance:")
print(f"✅ Accuracy:  {accuracy:.2f}%")
print(f"✅ Precision: {precision:.2f}%")
print(f"✅ Recall:    {recall:.2f}%")
print(f"✅ F1-score:  {f1:.2f}%")

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y, y_pred_mapped))

print("\n📝 Detailed Classification Report:")
print(classification_report(y, y_pred_mapped, target_names=["Clean", "Trojan"]))


Missing values before handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rolloff      0
spectral_entropy      0
spectral_contrast     1
dtype: int64
Missing values after handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rolloff      0
spectral_e

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# ------------------------------
# 1. Check for GPU
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Using device: {device}")

# ------------------------------
# 2. Load the Dataset
# ------------------------------
file_path = "IL_T600_cleaned.csv"
if not os.path.exists(file_path):
    print(f"❌ ERROR: File '{file_path}' not found! Place it in the same directory.")
    exit(1)

df = pd.read_csv(file_path)

# Rename 'Label' to 'label' if needed
df.rename(columns={'Label': 'label'}, inplace=True)

# ------------------------------
# 3. Handle Missing Values
# ------------------------------
print("Missing values per column before handling:\n", df.isnull().sum())

# Fill NaN values with column mean
df.fillna(df.mean(), inplace=True)

# Optional: Drop rows with any remaining NaNs
df.dropna(inplace=True)

print("Missing values per column after handling:\n", df.isnull().sum())

# ------------------------------
# 4. Separate Features & Labels
# ------------------------------
if 'label' not in df.columns:
    raise ValueError("❌ 'label' column is missing from the CSV. Please check your data.")

X = df.drop(columns=['label'])
y = df['label'].values  # 0 = Normal, 1 = Trojan (assuming binary)

# ------------------------------
# 5. Normalize Features
# ------------------------------
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features normalized.")

# ------------------------------
# 6. Build the Graph Edges
# ------------------------------
# (A) Temporal adjacency: connect each time step i -> i+1 (and i+1 -> i for undirected)
temporal_edges = []
num_samples = len(X_scaled)
for i in range(num_samples - 1):
    temporal_edges.append([i, i+1])
    temporal_edges.append([i+1, i])  # comment out if you prefer a directed chain

temporal_edges = np.array(temporal_edges).T  # shape (2, 2*(num_samples-1))

# (B) KNN adjacency
k_neighbors = 5  # Adjust based on dataset size
knn_graph = kneighbors_graph(X_scaled, k_neighbors, mode="connectivity", include_self=False)
knn_coo = knn_graph.tocoo()
knn_edges = np.vstack([knn_coo.row, knn_coo.col])  # shape (2, number_of_knn_edges)

# Combine edges
all_edges = np.concatenate([temporal_edges, knn_edges], axis=1)
edge_index = torch.tensor(all_edges, dtype=torch.long)

# ------------------------------
# 7. Create PyTorch Geometric Data
# ------------------------------
x_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor).to(device)
print("✅ Graph dataset created with temporal + KNN edges.")

# ------------------------------
# 8. Train/Test Split
# ------------------------------
# Randomly assign ~80% of nodes to training, 20% to testing
mask = torch.rand(num_samples).to(device)
train_mask = mask < 0.8
test_mask = ~train_mask

print(f"\n✅ Training samples: {train_mask.sum().item()}")
print(f"✅ Testing samples: {test_mask.sum().item()}")

# ------------------------------
# 9. Define GNN Model
# ------------------------------
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, output_dim=2):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# Initialize model, optimizer, loss
model = GNN(input_dim=X.shape[1], hidden_dim=256, output_dim=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
print("✅ GNN model initialized.")

# ------------------------------
# 10. Train the Model
# ------------------------------
epochs = 2000
print("\n✅ Training Started...")

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    out = model(data)  # shape: [num_samples, 2]
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # Compute training accuracy
    train_pred = out[train_mask].argmax(dim=1)
    train_acc = (train_pred == data.y[train_mask]).float().mean().item()

    if (epoch + 1) % 100 == 0:  # Print every 100 epochs
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.2%}")

print("\n✅ Training completed.")

# ------------------------------
# 11. Save the Model
# ------------------------------
torch.save(model.state_dict(), "trojan_gnn_model.pth")
print("\n✅ Model saved as 'trojan_gnn_model.pth'.")

# ------------------------------
# 12. Evaluate the Model
# ------------------------------
from sklearn.metrics import roc_auc_score               # <-- add this import

# ------------------------------
# 12. Evaluate the Model
# ------------------------------
model.eval()
with torch.no_grad():
    out = model(data)                                   # logits, shape [N, 2]

    # ─── NEW: probability that each node is Trojan (class-1) ───
    y_score = torch.softmax(out[test_mask], dim=1)[:, 1].cpu().numpy()

    test_pred = out[test_mask].argmax(dim=1)

# Compute Metrics
y_true = data.y[test_mask].cpu().numpy()
y_pred = test_pred.cpu().numpy()

auc      = roc_auc_score(y_true, y_score) * 100         # <-- NEW
accuracy = accuracy_score(y_true, y_pred) * 100
precision = precision_score(y_true, y_pred, zero_division=0) * 100
recall    = recall_score(y_true, y_pred, zero_division=0) * 100
f1        = f1_score(y_true, y_pred, zero_division=0) * 100

# Print Results
print("\n🎯 Model Performance on Test Nodes:")
print(f"✅ AUC:       {auc:.2f}%")                      # <-- NEW
print(f"✅ Accuracy:  {accuracy:.2f}%")
print(f"✅ Precision: {precision:.2f}%")
print(f"✅ Recall:    {recall:.2f}%")
print(f"✅ F1-score:  {f1:.2f}%")




✅ Using device: cuda
Missing values per column before handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatness     0
spectral_rolloff      0
spectral_entropy      0
spectral_contrast     1
dtype: int64
Missing values per column after handling:
 label                 0
Mean                  0
RMS                   0
Variance              0
Standard Deviation    0
Peak-to-Peak          0
Crest Factor          0
Skewness              0
Kurtosis              0
Energy                0
Entropy               0
Max                   0
Min                   0
peak_magnitude        0
spectral_centroid     0
spectral_bandwidth    0
spectral_flatn